## Neural Part Of Speech Tagging

Теперь мы собираемся решить ту же проблему тегирования POS с помощью нейронных сетей. 
<img src=https://i.stack.imgur.com/6pdIT.png width=320>

С точки зрения глубокого обучения это задача прогнозирования последовательности выходных данных, выровненной с последовательностью входных данных. Есть несколько задач, соответствующих этой формулировке:
* Маркировка частей речи - вспомогательная задача для многих проблем НЛП
* Распознавание именованных сущностей — для чат-ботов и поисковых роботов
* Предсказание структуры белка - для биоинформатики 

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import nltk
import sys
import numpy as np

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
print(data[0])

[('the', 'DET'), ('fulton', 'NOUN'), ('county', 'NOUN'), ('grand', 'ADJ'), ('jury', 'NOUN'), ('said', 'VERB'), ('friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]


In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [4]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))
    
    
draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


### Построение словарей

Как и раньше, нам нужно построить отображение токенов на целочисленные идентификаторы. На этот раз наша модель работает на уровне слов, обрабатывая одно слово за шаг RNN. Это означает, что нам придется иметь дело с гораздо большим словарным запасом.

К счастью для нас, мы получаем эти слова только в качестве входных данных, то есть нам не нужно их предсказывать. Это означает, что мы можем иметь большой словарный запас бесплатно, используя встраивание слов. 

In [5]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

# Давайте измерим, какая часть слов данных находится в словаре 
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [6]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

Конвертируем слова и теги в матрицу фиксированного размера

In [7]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""
    
    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix



In [8]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

### Построим модель

В отличие от нашей предыдущей лаборатории, на этот раз мы сосредоточимся на высокоуровневом интерфейсе keras для рекуррентных нейронных сетей. Это настолько просто, насколько вы можете получить с помощью RNN, хотя и несколько ограничено для сложных задач, таких как seq2seq.

По умолчанию все keras RNN применяются ко всей последовательности входных данных и создают последовательность скрытых состояний `(return_sequences=True` или только последнее скрытое состояние `(return_sequences=False)`. Все повторения происходят под капотом.

В верхней части нашей модели нам нужно применить плотный слой к каждому временному шагу независимо. На данный момент keras.layers.Dense по умолчанию применяется один раз ко всем объединенным временным шагам. Мы используем __keras.layers.TimeDistributed__ для изменения плотного слоя, чтобы он применялся как к пакетной, так и к временной осям. 

In [9]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

# Добавим верхний слой, который предсказывает вероятности тегов 
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


__Training:__ в этом случае мы не хотим заранее подготавливать весь набор обучающих данных. Основная причина в том, что длина каждого пакета зависит от максимальной длины предложения в пакете. Это оставляет нам два варианта: использовать собственный обучающий код, как на предыдущем семинаре, или использовать генераторы.

Модели Keras имеют метод __`model.fit_generator`__, который принимает генератор Python, выдающий по одному пакету за раз. Но сначала нам нужно реализовать такой генератор: 

In [10]:
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot
        

__Callbacks:__ Еще одна вещь, которая нам нужна, — это измерение производительности модели. Сложность заключается не в том, чтобы считать точность после окончания предложения (при заполнении), а в том, чтобы убедиться, что мы считаем все данные проверки ровно один раз.

Хотя нет ничего невозможного в том, чтобы убедить Keras сделать все это, мы можем также написать собственный обратный вызов, который сделает это.
Обратные вызовы Keras позволяют вам написать собственный код, который будет запускаться один раз в каждую эпоху или каждый мини-пакет. Мы определим его через LambdaCallback 

In [11]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    # Предскажем теговые вероятности формы  [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    # Вычислим accuracy, исключая padding 
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()
        

In [12]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5
1344/1343 [==============================] - 38s 29ms/step - loss: 0.2523

Measuring validation accuracy...
14335/14335 [==============================] - 6s 413us/step

Validation accuracy: 0.94044

Epoch 2/5
1344/1343 [==============================] - 36s 27ms/step - loss: 0.0585

Measuring validation accuracy...
14335/14335 [==============================] - 6s 400us/step

Validation accuracy: 0.94423

Epoch 3/5
1344/1343 [==============================] - 36s 27ms/step - loss: 0.0515

Measuring validation accuracy...
14335/14335 [==============================] - 6s 406us/step

Validation accuracy: 0.94559

Epoch 4/5
1344/1343 [==============================] - 35s 26ms/step - loss: 0.0469

Measuring validation accuracy...
14335/14335 [==============================] - 6s 405us/step

Validation accuracy: 0.94570

Epoch 5/5
1344/1343 [==============================] - 35s 26ms/s

Измерим окончательную accuracy на всем тестовом наборе. 

In [13]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

14335/14335 [==============================] - 6s 391us/step
Final accuracy: 0.94546


### Bidirectional

Поскольку мы анализируем полную последовательность, мы можем смотреть на будущие данные.

Простой способ добиться этого — двигаться в обоих направлениях одновременно, создавая __bidirectional RNN__.

В Keras вы можете добиться этого как вручную (используя два LSTM и Concatenate), так и с помощью __`keras.layers.Bidirectional`__.

Он работает так же, как `TimeDistributed`, который мы видели раньше: вы оборачиваете его вокруг рекуррентного слоя (сейчас SimpleRNN, а позже LSTM/GRU), и он фактически создает два слоя под капотом.

Ваша первая задача — использовать такой слой, как наш POS-тегер. 

In [14]:
# Определите модель, которая использует двунаправленный SimpleRNN 
model_1 = keras.models.Sequential()

model_1 = keras.models.Sequential()
model_1.add(L.InputLayer([None],dtype='int32'))
model_1.add(L.Embedding(len(all_words),50))

bidir = L.SimpleRNN(64, return_sequences=True)
bidir = L.Bidirectional(bidir)
model_1.add(bidir)

# Добавим верхний слой, который предсказывает вероятности тегов 
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model_1.add(stepwise_dense)


In [15]:
model_1.compile('adam','categorical_crossentropy')

model_1.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1344/1343 [==============================] - 66s 49ms/step - loss: 0.1926

Measuring validation accuracy...
14335/14335 [==============================] - 11s 749us/step

Validation accuracy: 0.95573

Epoch 2/5
1344/1343 [==============================] - 66s 49ms/step - loss: 0.0426

Measuring validation accuracy...
14335/14335 [==============================] - 11s 745us/step

Validation accuracy: 0.96084

Epoch 3/5
1344/1343 [==============================] - 65s 48ms/step - loss: 0.0354

Measuring validation accuracy...
14335/14335 [==============================] - 11s 738us/step

Validation accuracy: 0.96236

Epoch 4/5
1344/1343 [==============================] - 65s 48ms/step - loss: 0.0302

Measuring validation accuracy...
14335/14335 [==============================] - 11s 744us/step

Validation accuracy: 0.96220

Epoch 5/5
1344/1343 [==============================] - 65s 48ms/step - loss: 0.0256

Measuring validation accuracy...
14335/14335 [=========================

In [16]:
acc = compute_test_accuracy(model_1)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

14335/14335 [==============================] - 11s 749us/step

Final accuracy: 0.96255
Well done!


Задача I: Структурированные функции потерь (больше бонусных баллов)

Поскольку мы помечаем всю последовательность сразу, мы могли бы также научить нашу сеть делать это. Помните линейную CRF из лекции? Вы также можете использовать его как функцию потерь для вашей RNN.


   * Есть несколько способов сделать это, но мы рекомендуем начать с [условных случайных полей](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)
   * Вы можете подключить CRF в качестве функции потерь и по-прежнему тренироваться с помощью обратного распространения. Для вас даже есть аккуратная [реализация] tensorflow (https://www.tensorflow.org/addons/api_docs/python/tfa/layers/CRF).
   * Кроме того, вы можете обусловить свою модель предыдущими тегами (сделать ее авторегрессивной) и выполнить __beam search__ по этой модели. 

In [22]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-5lh1mdg3
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-5lh1mdg3
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=3ea0ab7d0fac9054c9628cf0be60e7895417fe5d12c0a743e7c9bf770f741e77
  Stored in directory: /tmp/pip-ephem-wheel-cache-a4gvo389/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [29]:
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy

In [31]:
# Определите модель, которая использует двунаправленный SimpleRNN 

model_crf = keras.models.Sequential()
model_crf.add(L.InputLayer([None],dtype='int32'))
model_crf.add(L.Embedding(len(all_words),50))

bidir = L.SimpleRNN(64, return_sequences=True)
bidir = L.Bidirectional(bidir)
model_crf.add(bidir)

# Добавим верхний слой, который предсказывает вероятности тегов 
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
# stepwise_dense = L.TimeDistributed(stepwise_dense)
model_crf.add(stepwise_dense)

In [32]:
model_crf.compile(optimizer="adam", loss=crf_loss, metrics=[crf_accuracy])

model_crf.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

AttributeError: ignored

#### Несколько советов
Вот еще несколько советов о том, как улучшить тренировки, которые немного сложнее реализовать. Мы настоятельно рекомендуем вам попробовать их _после_ того, как вы получите хорошую начальную модель.
* __Используйте предварительно обученные встраивания__: вы можете использовать предварительно обученные веса из [оттуда](http://ahogrammer.com/2017/01/20/the-list-of-pretrained-word-embeddings/), чтобы запустить встраивание слой.
  * Слой внедрения имеет матрицу W (layer.W), которая содержит вложения слов для каждого слова в словаре. Вы можете просто перезаписать их с помощью tf.assign.
  * При использовании предварительно обученных эмбеддингов обратите внимание на то, что словарь модели отличается от вашего собственного.
  * Вы можете переключить обучаемый = False для встраивания слоя в первые несколько эпох, как при обычной тонкой настройке.
* __Выходите за рамки SimpleRNN__: есть keras.layers.LSTM и keras.layers.GRU.
  * Если вы хотите использовать пользовательскую рекуррентную ячейку, прочитайте [это](https://keras.io/layers/recurrent/#rnn)
  * Вы также можете использовать одномерные свертки (`keras.layers.Conv1D`). Они часто так же хороши, как повторяющиеся слои, но с меньшим переоснащением.
* __Сложите больше слоев__: если есть общий мотив для этого курса, то это наложение слоев друг на друга
  * Вы можете просто добавить слои recurent и 1dconv друг на друга, и keras это поймет
  * Просто помните, что более крупным сетям может потребоваться больше эпох для обучения
* __Regularization__: вы можете применять отсев как обычно, но также и в соответствии со специфическими для RNN способами.
  * `keras.layers.Dropout` работает между слоями RNN
  * Повторяющиеся слои также имеют параметр recurrent_dropout.
* __Gradient clipping__: Если ваши тренировки не так стабильны, как вам хотелось бы, установите `clipnorm` в вашем оптимизаторе.
  * Другими словами, неплохо следить за кривой потерь для каждой мини-партии. Попробуйте обратный вызов tensorboard или что-то подобное.
* __Word Dropout__: tl;dr случайным образом заменяет слова на UNK во время обучения.
  * Это также может имитировать увеличение количества неизвестных слов в тестовом наборе.
* __Увеличенный словарный запас__: вы можете повысить производительность, расширив входной словарь вашей модели с 5000 до каждого слова!
  * Просто убедитесь, что ваша модель не подходит слишком много из-за большого количества параметров.
  * В сочетании с регуляризаторами или предварительно обученными векторами слов это может быть действительно хорошо, потому что сейчас наша модель слепа к> 5% слов.
* __Более эффективная пакетная обработка__: сейчас TF тратит много времени на перебор "0"
  * Это происходит потому, что пакет всегда дополняется до длины самого длинного предложения
  * Вы можете ускорить процесс, предварительно сгенерировав партии одинаковой длины и заполнив их случайно выбранной предварительно сгенерированной партией.
  * Технически это нарушает i.i.d. предположение, но это работает, если вы не придумали какую-нибудь безумную rnn-архитектуру.
* __Самый главный совет__: не впихивайте все сразу!
  * Если вы добавите много модификаций, некоторые из них почти неизбежно будут вредными, и вы никогда не узнаете, какие из них.
  * Вместо этого старайтесь делать небольшие итерации и записывать результаты экспериментов, чтобы ориентироваться в дальнейшем поиске.
    
Удачной охоты! 